In [ ]:
# Description: This script uses GPT-4o to detect and summarize hydrogen-related remarks in Korean National Assembly records and saves the results to an Excel file.

import pandas as pd
import openai
import re
from tqdm import tqdm
from google.colab import files

# 🔑 Step 1: Upload and load OpenAI API key
# 🔑 1단계: OpenAI API 키를 업로드하고 불러옵니다.
print("🔑 Please upload your OpenAI API key (.txt) file.")
uploaded = files.upload()
api_key_file = list(uploaded.keys())[0]

with open(api_key_file, "r") as f:
    openai.api_key = f.read().strip()

# 📂 Step 2: Upload National Assembly transcript data
# 📂 2단계: 국회 회의록 엑셀 파일을 업로드합니다.
print("📂 Please upload the National Assembly transcript (.xlsx) file.")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# ✅ Step 3: Load data
# ✅ 3단계: 엑셀 데이터를 불러옵니다.
df = pd.read_excel(file_path)

# ✅ Step 4: Rename columns and retain only required ones
# ✅ 4단계: 컬럼명을 정리하고 필요한 컬럼만 유지합니다.
column_mapping = {
    "키워드": "키워드",
    "회의번호": "회의번호",
    "회의구분": "회의구분",
    "위원회": "위원회",
    "연도": "연도",
    "안건": "안건",
    "발언자": "발언자",
    "발언내용": "발언내용"
}
df.rename(columns=column_mapping, inplace=True)
required_columns = ["키워드", "회의번호", "회의구분", "위원회", "연도", "안건", "발언자", "발언내용"]
df = df[required_columns]

# ✅ Step 5: Determine if a speech is hydrogen-related using GPT
# ✅ 5단계: GPT를 사용해 수소 관련 발언 여부를 판단합니다.
def is_hydrogen_related(text):
    try:
        prompt = f"""
다음 국회의원 발언이 다음 키워드 중 하나라도 관련이 있는지 판단해줘: '수소', '연료전지', '수소차', '수소경제'.
# Determine whether the following National Assembly remark is related to any of these keywords: 'hydrogen', 'fuel cell', 'hydrogen vehicle', 'hydrogen economy'.

- 단, 키워드가 포함되지 않더라도 맥락적으로 관련이 있다면 'YES'라고 해줘.
# Even if the keywords are not explicitly mentioned, respond with 'YES' if contextually related.

- 관련이 없으면 'NO'라고만 답변해줘.
# If the remark is clearly unrelated, respond with 'NO' only.

발언 내용: {text}
# Speech content
"""
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        result = response.choices[0].message.content.strip()
        return result  # "YES" 또는 "NO"
    except Exception as e:
        print(f"❌ GPT 호출 중 오류 발생: {e}")
        return "NO"

# ✅ Step 6: Apply GPT to classify hydrogen-related speeches
# ✅ 6단계: GPT를 활용하여 수소 관련 발언을 분류합니다.
df["GPT_Verified"] = df["발언내용"].apply(is_hydrogen_related)

# ✅ Step 7: Filter only "YES" speeches
# ✅ 7단계: 'YES'로 분류된 발언만 추출합니다.
df_filtered = df[df["GPT_Verified"].str.contains("YES", case=False, na=False)]

# ✅ Step 8: Summarize each hydrogen-related speech using GPT
# ✅ 8단계: GPT를 활용해 핵심 내용으로 발언을 요약합니다.
def summarize_text(text):
    try:
        prompt = f"""
다음 국회 발언을 핵심 내용만 유지하면서 요약해줘.
# Please summarize the following National Assembly remark while preserving only the core message.

✅ 요약 원칙:
# ✅ Summarization Guidelines:
- 발언자의 주요 주장(정책, 기술, 비용, 제도 개선 등)을 포함할 것.
# Include key claims such as policy, technology, cost, or institutional improvement.
- 불필요한 감탄사, 인사말, 반복 표현, 연결어는 제거할 것.
# Remove unnecessary exclamations, greetings, repetitions, and conjunctions.
- 단정적인 표현은 피하고, 원문의 불확실성은 유지할 것.
# Avoid overly assertive language and preserve original uncertainties.
- 새로운 정보를 추가하지 말 것.
# Do not introduce any new information not present in the original.
- 논점을 바꾸지 말 것.
# Do not alter the main topic or argument of the speech.
- 문장 구조는 유지하되 의미를 압축할 것.
# Maintain sentence structure but compress meaning where possible.
- 길이 제한 없이 간결하고 명확하게 작성할 것.
# No strict length limit, but keep it concise and clear.

💡 예제:
# 💡 Example:
📌 원문 발언:
"우리 인류는 메소포타미아 유역의 수메르 문명부터 5000년 넘게 도시를 만들고 도시를 발전시켜 왔다. 이제는 도시 계획에 있어서 탄소중립과 친환경 에너지를 고려해야 한다."
# Original remark: "Humanity has built and developed cities for over 5000 years since the Sumerian civilization. Now, we must consider carbon neutrality and eco-friendly energy in urban planning."
📌 잘못된 요약:
"도시의 본질은 수소가 아닌 전문기술과 자본으로 이루어진 것입니다. 현재 논의 중인 수소 정책은 이에 대한 해결 방안을 제시하고 있습니다." ❌
# Incorrect summary: Changes the topic to hydrogen, which was not in the original ❌
📌 올바른 요약:
"도시 계획에서는 탄소중립과 친환경 에너지를 고려해야 한다." ✅
# Correct summary: Focuses on original message of carbon neutrality ✅

📌 원문 발언:
"맥킨지의 분석을 보면 한국 수소산업 로드맵이 2050년도 최종 에너지 소비량의 20%, 전체 목표 대비 온실가스 감축량이 17%, 도로오염원의 30% 감소 등을 수소경제로 달성할 것이라고 한국의 수소산업을 전망한 바가 있습니다."
# Original remark: "McKinsey's analysis estimates that Korea's hydrogen roadmap will achieve 20% of final energy consumption, reduce GHG emissions by 17%, and cut road pollution by 30% by 2050."
📌 요약 결과:
"맥킨지의 분석에 따르면, 한국 수소산업 로드맵은 2050년까지 최종 에너지 소비량의 20%, 온실가스 감축량의 17%, 도로오염원의 30%를 수소경제로 달성할 수 있을 것으로 전망합니다."
# Summary: Accurately compresses figures and retains the forecast ✅

📌 원문 발언:
"부생수소는 아니고요 정확히 말씀드리면 부생수소가 있고, 지금 현재 수소를 만드는 방법이 여러 가지가 있습니다마는 우리나라에서는 부생수소하고 또 하나는 천연가스를 개질해서 만드는 방법이 있습니다."
# Original remark: "There are various methods to produce hydrogen. In Korea, we mainly use by-product hydrogen and reforming of natural gas."
📌 요약 결과:
"부생수소는 여러 가지 수소 생산 방법 중 하나입니다. 현재 우리나라에서는 주로 부생수소와 천연가스를 개질하여 수소를 생산합니다."
# Summary: Clear explanation of the main hydrogen production methods in Korea ✅

발언 내용: {text}
# Speech content
"""
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ GPT 요약 중 오류 발생: {e}")
        return text

# ✅ Step 9: Apply summarization with progress bar
# ✅ 9단계: 진행 표시줄과 함께 요약을 적용합니다.
tqdm.pandas()
df_filtered["Summarized_Speech"] = df_filtered["발언내용"].progress_apply(summarize_text)

# ✅ Step 10: Drop unnecessary columns
# ✅ 10단계: 불필요한 컬럼 제거
df_filtered = df_filtered.drop(columns=["발언내용", "GPT_Verified"])

# ✅ Step 11: Save and download the final Excel file
# ✅ 11단계: 최종 엑셀 파일 저장 및 다운로드
output_file = "gpt4o_hydrogen_related_speech_summary.xlsx"
df_filtered.to_excel(output_file, index=False, engine="openpyxl")
files.download(output_file)

print(f"✅ Processing complete! Download file: {output_file}")


🔑 OpenAI API 키(.txt) 파일을 업로드하세요.


Saving 가영 API for 논문.txt to 가영 API for 논문.txt
📂 회의록 데이터(.xlsx) 파일을 업로드하세요.


Saving 수소_연료전지_통합_2024만_샘플 제외.xlsx to 수소_연료전지_통합_2024만_샘플 제외.xlsx


100%|██████████| 205/205 [10:31<00:00,  3.08s/it]
<ipython-input-3-2ea6e1742854>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["요약된 발언"] = df_filtered["발언내용"].progress_apply(summarize_text)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 전처리 완료! 다운로드 링크: processed_national_assembly_speeches.xlsx
